# Stock Management System - Optimización de Producción

Este notebook implementa un sistema para gestionar el inventario y optimizar la producción utilizando programación lineal.

In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from typing import Dict, Any

In [ ]:
# Paso 1: Cargar los datos
print("Paso 1: Cargar los datos")
data_path = "stock_data.csv"
try:
    # Leer el archivo CSV
    df = pd.read_csv(data_path, sep=';', encoding='latin1', skiprows=3, header=0)
    df.columns = df.iloc[0]  # Usar la primera fila como encabezado
    df = df[1:]  # Eliminar la fila usada como encabezado
    df.columns = df.columns.str.strip()  # Limpiar espacios
    print("Datos cargados correctamente.")
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")

In [ ]:
# Paso 2: Limpiar y preparar los datos
print("\nPaso 2: Limpiar y preparar los datos")
try:
    # Eliminar filas vacías
    df.dropna(how='all', inplace=True)

    # Convertir columnas relevantes a numéricas
    num_columns = ['Cj/H', 'Disponible', 'Calidad', 'Stock Externo', 'M_Vta -15', 'M_Vta -15 AA']
    for col in num_columns:
        df[col] = pd.to_numeric(df[col].replace({',': '.'}, regex=True), errors='coerce')

    # Reemplazar valores nulos
    df.fillna({
        'Cj/H': 1,  # Producción mínima
        'Disponible': 0,
        'Calidad': 0,
        'Stock Externo': 0,
        'M_Vta -15': 0,
        'M_Vta -15 AA': 1  # Evitar divisiones por 0
    }, inplace=True)

    # Calcular el stock total
    df['STOCK_TOTAL'] = df['Disponible'] + df['Calidad'] + df['Stock Externo']
    print("Datos limpiados y preparados.")
except KeyError as e:
    print(f"Error en las columnas del DataFrame: {e}")

In [ ]:
# Paso 3: Calcular la estimación de demanda
print("\nPaso 3: Calcular la estimación de demanda")
try:
    # Demanda base y variación anual
    df['DEMANDA_MEDIA'] = df['M_Vta -15']
    df['VAR_ANUAL'] = abs(1 - (df['M_Vta -15'] / df['M_Vta -15 AA'])) * 100

    # Ajustar demanda para variaciones significativas (>20%)
    mask = df['VAR_ANUAL'] > 20
    df.loc[mask, 'DEMANDA_MEDIA'] *= (1 + df.loc[mask, 'VAR_ANUAL'] / 100)
    print("Estimación de demanda calculada.")
except Exception as e:
    print(f"Error al calcular la estimación de demanda: {e}")

In [ ]:
# Paso 4: Optimizar la producción
print("\nPaso 4: Optimizar la producción")
try:
    available_hours = 100
    maintenance_hours = 5
    pending_orders = {"244719": 50, "274756": 30}

    # Calcular horas disponibles netas
    net_hours = available_hours - maintenance_hours

    # Parámetros básicos
    production_rates = df['Cj/H'].values
    current_stock = df['STOCK_TOTAL'].values
    demand = df['DEMANDA_MEDIA'].values

    # Validaciones previas
    if net_hours <= 0:
        raise ValueError("No hay horas disponibles para producción.")

    if (current_stock < demand).all():
        print("Advertencia: Stock inicial insuficiente para satisfacer la demanda.")

    # Diagnóstico
    print("Stock inicial:", current_stock)
    print("Demanda estimada:", demand)
    print("Tasas de producción:", production_rates)
    print("Horas disponibles netas:", net_hours)

    # Definir función objetivo: maximizar producción efectiva
    c = -1 * np.minimum(demand, current_stock)

    # Restricciones
    A_ub = []
    b_ub = []

    # Restricción 1: Horas totales disponibles
    A_ub.append(1 / production_rates)
    b_ub.append(net_hours)

    # Restricción 2: Stock de seguridad
    safety_stock = np.minimum(demand * 3, current_stock)  # SAFETY_STOCK_DAYS = 3
    for i in range(len(production_rates)):
        row = np.zeros(len(production_rates))
        row[i] = -1
        A_ub.append(row)
        b_ub.append(-(safety_stock[i] - current_stock[i]))

    # Convertir restricciones a arrays numpy
    A_ub = np.array(A_ub, dtype=object)
    b_ub = np.array(b_ub)

    # Resolver el problema de optimización
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=(0, None), method='highs')

    if not result.success:
        print("Error en la optimización:", result.message)
        raise ValueError("Optimización no exitosa")

    # Crear plan de producción
    df['CAJAS_PRODUCIR'] = result.x
    df['HORAS_PRODUCCION'] = result.x / df['Cj/H']

    print("Optimización completada.")
except Exception as e:
    print(f"Error al optimizar la producción: {e}")

In [ ]:
def optimize_production(self, available_hours: float, maintenance_hours: float, pending_orders: Dict[str, float] = None):
    try:
        # Ajustar horas disponibles
        net_hours = available_hours - maintenance_hours
        production_rates = self.df['Cj/H'].fillna(1).values
        current_stock = self.df['STOCK_TOTAL'].values
        demand = self.df['DEMANDA_MEDIA'].values

        # Función objetivo
        c = -1 * np.maximum(demand - current_stock, 0)

        # Restricciones
        A_ub = [1 / np.where(production_rates == 0, 1, production_rates)]
        b_ub = [net_hours]

        min_production = self.MIN_PRODUCTION_HOURS * production_rates
        A_ub.extend(-1 * np.eye(len(production_rates)))
        b_ub.extend(-1 * min_production)

        # Resolver problema de optimización
        result = linprog(
            c,
            A_ub=np.array(A_ub),
            b_ub=np.array(b_ub),
            bounds=(0, None),
            method='highs'
        )

        # Crear plan de producción
        production_plan = self.df.copy()
        production_plan['CAJAS_PRODUCIR'] = result.x
        production_plan['HORAS_PRODUCCION'] = np.divide(
            result.x, 
            production_plan['Cj/H'].fillna(1),
            out=np.zeros_like(result.x),
            where=production_plan['Cj/H'].fillna(1) > 0
        )

        print("Plan de producción generado:")
        print(production_plan[['COD_ART', 'CAJAS_PRODUCIR', 'HORAS_PRODUCCION']].head())
        return production_plan
    except Exception as e:
        raise Exception(f"Error al optimizar la producción: {e}")


In [ ]:
production_plan = system.optimize_production(100, 5)


In [ ]:
def display_summary_table(self, production_plan: pd.DataFrame):
    try:
        summary_table = production_plan[['COD_ART', 'NOM_ART', 'CAJAS_PRODUCIR', 'HORAS_PRODUCCION', 'STOCK_TOTAL']].head(10)
        print("\nResumen del plan de producción:\n")
        print(summary_table)
    except Exception as e:
        raise Exception(f"Error al mostrar la tabla resumen: {e}")


In [ ]:
system.display_summary_table(production_plan)
